In [ ]:
import onnxruntime as ort
import pandas as pd
import numpy as np
import joblib

class AnomalyDetection:
    
    def __init__(self, model_path='model/lstm_autoencoder.onnx', 
                       scaler_path='model/scaler.pkl', 
                       threshold_path='model/threshold.pkl'):
        # ONNX 모델, 스케일러, 임계값 로드
        self.model = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
        self.scaler = joblib.load(scaler_path)
        self.threshold = joblib.load(threshold_path)

    def predict(self, x):
        """
        예측 함수: 주어진 데이터를 ONNX 모델을 통해 예측하고, 이상 여부를 판단합니다.
        
        :param x: 입력 데이터 (배치 크기, timesteps, features)
        :return: 이상 여부 (0: 정상, 1: 이상)
        """
        # 데이터를 스케일링
        x_scaled = self.scale(x)
        
        # ONNX 모델을 통한 예측
        input_name = self.model.get_inputs()[0].name
        pred = self.model.run(None, {input_name: x_scaled.astype(np.float32)})[0]
        
        # MSE 계산
        mse = np.mean(np.power(self.flatten(x) - self.flatten(pred), 2), axis=1)
        
        # 실제값 (True_class)
        true_y = x[:, -1, 0]  # 마지막 타임스텝의 첫 번째 feature 값을 실제값으로 사용
        
        # 데이터프레임으로 출력
        result_df = pd.DataFrame({
            'True_class': true_y,
            'Predicted_class': pred,
            'Reconstruction_error': mse,
            'Threshold': self.threshold,
            'Anomaly': ['Yes' if e == 1 else 'No' for e in pred]
        })
        
        return result_df

    def scale(self, X):
        """
        스케일링 함수: 입력 데이터를 미리 학습된 스케일러로 변환합니다.
        
        :param X: 입력 데이터 (배치 크기, timesteps, features)
        :return: 스케일링된 데이터
        """
        X_scaled = X.copy()
        for i in range(X.shape[0]):
            X_scaled[i, :, :] = self.scaler.transform(X[i, :, :])
        return X_scaled

    def flatten(self, X):
        """
        플래튼 함수: 시계열 데이터를 1D로 변환하여 MSE 계산에 적합하게 만듭니다.
        
        :param X: 입력 데이터 (배치 크기, timesteps, features)
        :return: 평탄화된 데이터
        """
        flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
        for i in range(X.shape[0]):
            flattened_X[i] = X[i, (X.shape[1]-1), :]  # 마지막 timesteps에서 feature 추출
        return flattened_X

In [ ]:
import streamlit as st
import pandas as pd

# Streamlit 앱 구현
st.title("설비 이상 탐지 대시보드")

# 파일 업로드
uploaded_file = st.file_uploader("CSV 파일 업로드", type="csv")

if uploaded_file is not None:
    # CSV 파일을 Pandas DataFrame으로 읽기
    data = pd.read_csv(uploaded_file)
    
    # 데이터를 Numpy 배열로 변환 (예: (배치 크기, timesteps, feature 수))
    X_input = data.values.reshape(-1, 10, 1)  # 예시로 timesteps=10, feature 수=1로 가정

    # AnomalyDetection 인스턴스 생성 및 예측
    anomaly_detector = AnomalyDetection()
    result_df = anomaly_detector.predict(X_input)

    # 예측 결과 테이블 표시
    st.write("### 예측 결과", result_df)

    # 예측된 이상/정상 분류의 비율
    st.write("### 이상/정상 분포")
    st.bar_chart(result_df['Anomaly'].value_counts())

    # reconstruction error 그래프
    st.write("### Reconstruction Error 분포")
    st.line_chart(result_df['Reconstruction_error'])

    # 임계값 표시
    st.write("### 설정된 임계값")
    st.write(anomaly_detector.threshold)

2025-04-26 17:19:54.880 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 17:19:54.914 
  command:

    streamlit run /home/so/.conda/envs/so/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-26 17:19:54.914 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 17:19:54.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 17:19:54.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 17:19:55.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 17:19:55.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [5]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 32.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 70.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.8 MB/s eta 0:00:00
